## Load libraries

In [ ]:
import multitaper.mtspec as mtspec
import multitaper.utils as utils
#import multitaper.mtcross as mtcross
import numpy as np
import matplotlib.pyplot as plt

## Load building data

In [ ]:
data     = utils.get_data('crisanto_mesetas.dat')
dt       = 1/200.
npts,ntr = np.shape(data)

print('npts, # of traces, dt ',npts, ntr, dt)

## Plot time series

In [ ]:
# create time vector
qtime = 3*60+55   # in seconds
t    = np.arange(npts)*dt - qtime
dmax = np.amax(data)/2

fig = plt.figure()
ax  = fig.add_subplot()
for i in range(ntr):
    ax.plot(t/60,data[:,i]/dmax+(i*3)+2,'k')
ax.set_xlabel('Time since M6.0 earthquake (min)')
ax.set_ylabel('Floor')
ax.set_yticks([2, 5, 8, 11, 14]);
ax.plot((0,0),(1,16),'r--')
ax.plot((1.66,1.66),(1,16),'r--')
fig = plt.figure()
ax  = fig.add_subplot()
for i in range(ntr):
    ax.plot(t,data[:,i]/dmax+(i*3)+2,'k')
ax.set_xlabel('Time since M6.0 earthquake (s)')
ax.set_xlim(0,100)
ax.set_ylabel('Floor')
ax.set_yticks([2, 5, 8, 11, 14]);
#plt.savefig('cr_spectrogram00.jpg')

In [ ]:
nw = 3.5
itrace = 4
x = data[:,itrace]
t2,freq,QIspec,MTspec = mtspec.spectrogram(x,dt,twin=20.,olap=0.5,
                               nw=nw,kspec=5,fmin=0.05,fmax=20.)

# Plot spectrogram and time series

In [ ]:
tspec = (t2-qtime)/60.     

df   = freq[2]-freq[1]
fnw  = nw*df
lfnw = np.array((-fnw,+fnw))

fl_1 = 3.0
fh_1 = 4.5
fl_2 = 0.7
fh_2 = 1.6

iloc1 = np.where((freq>=fl_1) & (freq<=fh_1))[0]
iloc2 = np.where((freq>=fl_2) & (freq<=fh_2))[0]

tbox  = np.array((np.min(tspec),np.max(tspec),
                 np.max(tspec),np.min(tspec),np.min(tspec)))
f2box = np.array((fl_2,fl_2,fh_2,fh_2,fl_2))
f1box = np.array((fl_1,fl_1,fh_1,fh_1,fl_1))

fig = plt.figure(figsize=(10, 6)) 
ax2 = fig.add_subplot(2,1,1)
ax2.plot(t/60,data[:,itrace]/dmax,'k')
ax2.set_xlim(np.min(tspec),np.max(tspec))
ax2.set_xticks([])
ax2.set_yticks([])
box    = ax2.get_position()
box.y0 = box.y0 + 0.24
ax2.set_position(box)

ax1 = fig.add_subplot(2,1,2)
X,Y = np.meshgrid(tspec,freq)
c = ax1.pcolor(X,Y,np.log10(QIspec),cmap='Greys', shading='auto',)
        # vmax=1e7)

ax1.plot(tbox,f2box,'w--',linewidth=1)
ax1.plot(tbox,f1box,'w--',linewidth=1)
ax1.set_xlabel('Time after M6.0 earthquake (min)')
ax1.set_ylabel('Frequency (Hz)')
ax1.set_ylim((0.05,15))
box    = ax1.get_position()
box.y1 = box.y1 + 0.30
ax1.set_position(box)

plt.savefig('figures/tf_waveforms.jpg')

# Zoomed spectrogram

In [ ]:
#------------------------------------------
# Plot spectrograms zoomed
#------------------------------------------
props = dict(facecolor='white', alpha=1.0)

f1    = freq[iloc1]
X1,Y1 = np.meshgrid(tspec,f1)
s1    = QIspec[iloc1,:]
s1b   = MTspec[iloc1,:]
for i in range(len(tspec)):
    s1[:,i]  = s1[:,i] /np.max(s1[:,i])
    s1b[:,i] = s1b[:,i]/np.max(s1b[:,i])
f2  = freq[iloc2]
X,Y = np.meshgrid(tspec,f2)
s2  = QIspec[iloc2,:]
s2b = MTspec[iloc2,:]
for i in range(len(tspec)):
    s2[:,i]  = s2[:,i] /np.max(s2[:,i])
    s2b[:,i] = s2b[:,i]/np.max(s2b[:,i])

fig = plt.figure(figsize=(10, 6)) 
ax1 = fig.add_subplot(2,2,1)
c = ax1.pcolor(X1,Y1,(s1),cmap='binary', shading='auto',vmin=0.1)
ax1.set_xticks([])
ax1.set_ylabel('Frequency (Hz)')
ax1.text(-3.5, 3.08, 'Quadratic',bbox=props, fontsize=10)

ax2 = fig.add_subplot(2,2,3)
c = ax2.pcolor(X,Y,(s2),cmap='binary', shading='auto',vmin=0.1)
ax2.set_ylabel('Frequency (Hz)')
ax2.set_xlabel('Time after M6.0 earthquake (min)')
ax2.text(-3.5, 0.75, 'Quadratic',bbox=props, fontsize=10)
box    = ax2.get_position()
box.y0 = box.y0 + 0.05
box.y1 = box.y1 + 0.05
ax2.set_position(box)


ax3 = fig.add_subplot(2,2,2)
c = ax3.pcolor(X1,Y1,(s1b),cmap='binary', shading='auto',vmin=0.1)
ax3.set_xticks([])
ax3.set_yticks([])
ax3.text(-3.5, 3.08, 'Adaptive',bbox=props, fontsize=10)
box    = ax3.get_position()
box.x0 = box.x0 - 0.05
box.x1 = box.x1 - 0.05
ax3.set_position(box)

ax4 = fig.add_subplot(2,2,4)
c = ax4.pcolor(X,Y,(s2b),cmap='binary', shading='auto',vmin=0.1)
ax4.set_xlabel('Time after M6.0 earthquake (min)')
ax4.set_yticks([])
ax4.text(-3.5, 0.75, 'Adaptive',bbox=props, fontsize=10)

box    = ax4.get_position()
box.x0 = box.x0 - 0.05
box.x1 = box.x1 - 0.05
box.y0 = box.y0 + 0.05
box.y1 = box.y1 + 0.05
ax4.set_position(box)

plt.savefig('figures/tf_spec2.jpg')


# Temporal behavior of peaks


In [ ]:
#------------------------------------------
# Select time windows
#------------------------------------------
k1 = 10
k2 = 28
k3 = 44

fig2 = plt.figure(figsize=(6,8))
ax2 = fig2.add_subplot(2,1,2)
ax2.plot(freq,QIspec[:,k1]/np.max(QIspec[iloc1,k1]),'b',
         label=r'time=%5.2f'%(tspec[10]));
ax2.plot(freq,QIspec[:,k2]/np.max(QIspec[iloc1,k2]),'r',
        label=r'time=%5.2f'%(tspec[28]));
ax2.plot(freq,QIspec[:,k3]/np.max(QIspec[iloc1,k3]),'m',
        label=r'time=%5.2f'%(tspec[44]));
ax2.set_xlim(fl_1,fh_1)
ax2.set_ylim(0,1.1)
ax2.plot(lfnw+4.28,[0.8,0.8],'k')
ax2.text(4.25,0.82,'2W')
ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('PSD (normalized)')

ax3 = fig2.add_subplot(2,1,1)
ax3.plot(freq,QIspec[:,k1]/np.max(QIspec[iloc2,k1]),'b',
         label='Pre-event');
ax3.plot(freq,QIspec[:,k2]/np.max(QIspec[iloc2,k2]),'r',
        label='Co-event');
ax3.plot(freq,QIspec[:,k3]/np.max(QIspec[iloc2,k3]),'m',
        label='Post-event');
ax3.set_xlim(0.4,fh_2+0.2)
ax3.set_ylim(0,1.1)
ax3.plot(lfnw+1.6,[0.8,0.8],'k')
ax3.text(1.57,0.83,'2W')
ax3.legend(loc='upper left')
ax3.set_xlabel('Frequency (Hz)')
ax3.set_ylabel('PSD (normalized)')
plt.savefig('figures/tf_spec1.jpg')


